# Benchmark EDUC

This notebook aims to benchmark the economic dispatch and unit commitment problem.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

import ams.benchmarks as bp

# Configure AMS logger
ams.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-09 23:36:24
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                      
----------  -----------------------------
ltbams      0.9.10.post153.dev0+gc73b7f8a
andes       1.9.1.post100+g2b17faac      
cvxpy       1.5.3                        
gurobipy    11.0.3                       
mosek       10.2.6                       
piqp        0.4.2                        
pandapower  2.14.11                      
numba       0.60.0                       


In [2]:
%matplotlib inline

In [3]:
ams.config_logger(stream_level=50)

In [4]:
cases = [
    './cases/pjm5bus_uced.xlsx',
    './cases/ieee14_uced.xlsx',
    './cases/ieee39_uced.xlsx',
    './cases/npcc_uced.xlsx',
    './cases/wecc_uced.xlsx',
]

In [5]:
solvers=['GUROBI', 'MOSEK']

In [6]:
# run individual case to ensure everything is working
sp = ams.load(cases[0],
              setup=True, no_output=True, default_config=True)
pre_time, sol = bp.time_routine(sp, routine='ED', solvers=solvers)
print(pre_time)
print(sol)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25
{'ams_mats': 0.0007, 'ams_parse': 0.0058, 'ams_eval': 0.0064, 'ams_final': 0.0001, 'ams_postinit': 0.0001}
{'GUROBI': {'time': 0.0278, 'obj': 62.022999999999996}, 'MOSEK': {'time': 0.0308, 'obj': 62.0230000813157}}


In [7]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logging.getLogger('pandapower').setLevel(logging.ERROR)

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

time_data_ed = np.zeros((n_iters, n_cases, len(bp.cols_pre) + len(solvers)))
obj_data_ed = np.zeros((n_cases, len(solvers)))

time_data_uc = np.zeros((n_iters, n_cases, len(bp.cols_pre) + len(solvers)))
obj_data_uc = np.zeros((n_cases, len(solvers)))

In [8]:
# Run ED
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, no_output=True, default_config=True)
    for n_iter in range(n_iters):
        pre_time, sol = bp.time_routine(system, routine='ED',
                                        solvers=solvers,
                                        ignore_dpp=True)
        time_data_ed[n_iter, n_case, :len(pre_time)] = np.array([i for i in pre_time.values()])
        time_data_ed[n_iter, n_case, len(pre_time):] = np.array([sol[solver]['time'] for solver in solvers])
    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data_ed[n_case, :] = obj

Case: ./cases/pjm5bus_uced.xlsx
Case: ./cases/ieee14_uced.xlsx
Case: ./cases/ieee39_uced.xlsx
Case: ./cases/npcc_uced.xlsx
Case: ./cases/wecc_uced.xlsx


In [9]:
# Run UC
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, no_output=True, default_config=True)
    for n_iter in range(n_iters):
        pre_time, sol = bp.time_routine(system, routine='UC',
                                        solvers=solvers,
                                        ignore_dpp=True)
        time_data_ed[n_iter, n_case, :len(pre_time)] = np.array([i for i in pre_time.values()])
        time_data_ed[n_iter, n_case, len(pre_time):] = np.array([sol[solver]['time'] for solver in solvers])
    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data_ed[n_case, :] = obj

Case: ./cases/pjm5bus_uced.xlsx
Case: ./cases/ieee14_uced.xlsx
Case: ./cases/ieee39_uced.xlsx
Case: ./cases/npcc_uced.xlsx
Case: ./cases/wecc_uced.xlsx


In [10]:
# Process and display the results
case_names = [case.split('/')[-1].split('.')[0] for case in cases]

time_ed = pd.DataFrame(columns=bp.cols_pre + solvers, index=case_names)
for case in case_names:
    time_ed.loc[case] = time_data_ed[:, case_names.index(case), :].mean(axis=0)

time_uc = pd.DataFrame(columns=bp.cols_pre + solvers, index=case_names)
for case in case_names:
    time_uc.loc[case] = time_data_uc[:, case_names.index(case), :].mean(axis=0)

# scale to ms
time_ed *= 1000 
time_uc *= 1000

In [11]:
time_ed

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK
pjm5bus_uced,0.58,5.21,6.0,0.19,0.16,26.47,24.51
ieee14_uced,0.63,5.23,6.47,0.19,0.13,38.83,40.71
ieee39_uced,0.76,5.26,7.25,0.19,0.14,70.66,71.24
npcc_uced,1.6,5.23,16.47,0.34,0.21,353.19,338.87
wecc_uced,1.69,5.24,12.97,0.37,0.16,329.09,362.66


In [12]:
time_uc

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK
pjm5bus_uced,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ieee14_uced,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ieee39_uced,0.0,0.0,0.0,0.0,0.0,0.0,0.0
npcc_uced,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wecc_uced,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
time_ed.to_csv('../results/results_ed.csv')
time_uc.to_csv('../results/results_uc.csv')